In [ ]:
%store -r __importRegression

In [ ]:
__importRegression

In [ ]:
air_store_info = pd.read_csv('input/air_store_info.csv')
train = pd.read_csv('input/air_visit_data.csv')
date_info = pd.read_csv('input/date_info.csv')
test = pd.read_csv('input/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
date_info.head()

In [ ]:
date_info.drop('day_of_week', axis=1, inplace=True)

## Extract air_store_id and visit_date from test

In [ ]:
test['air_store_id'] = test.id.apply(lambda x: str(x.split('_')[0]) + '_' + str(x.split('_')[1]))

In [ ]:
test['visit_date'] = test.id.apply(lambda x: str(x.split('_')[2]))

In [ ]:
test.head()

## Add holiday feature

In [ ]:
train = pd.merge(train, date_info, left_on='visit_date', right_on='calendar_date', how='left')

In [ ]:
train.head()

In [ ]:
test = pd.merge(test, date_info, left_on='visit_date', right_on='calendar_date', how='left')

In [ ]:
test.head()

## Add features from store info data

In [ ]:
air_store_info.head()

In [ ]:
train = pd.merge(train, air_store_info, on='air_store_id', how='left')

In [ ]:
train.head()

In [ ]:
test = pd.merge(test, air_store_info, on='air_store_id', how='left')

In [ ]:
test.head()

## Extract week count since 2016

In [ ]:
train['visit_date'] = pd.to_datetime(train.visit_date, format='%Y-%m-%d')

In [ ]:
test['visit_date'] = pd.to_datetime(test.visit_date, format='%Y-%m-%d')

In [ ]:
def weekSince2016(t):
    if t.year == 2016:
        return t.week
    else:
        return 52 + t.week

In [ ]:
test['week_count'] = test.visit_date.apply(weekSince2016)

In [ ]:
test.head()

In [ ]:
train['week_count'] = train.visit_date.apply(weekSince2016)

In [ ]:
train.head()

## Label encode air_store_id

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

le.fit(train.air_store_id)

In [ ]:
train['store_id'] = le.transform(train.air_store_id)

In [ ]:
train.head()

In [ ]:
test['store_id'] = le.transform(test.air_store_id)

In [ ]:
test.head()

## Label encode air_genre_name

In [ ]:
le.fit(train.air_genre_name)

train['genre_le'] = le.transform(train.air_genre_name)
test['genre_le'] = le.transform(test.air_genre_name)

In [ ]:
train.head()

In [ ]:
test.head()

## Label encode air_area_name

In [ ]:
le.fit(train.air_area_name)

train['area_le'] = le.transform(train.air_area_name)
test['area_le'] = le.transform(test.air_area_name)

In [ ]:
train.head()

In [ ]:
test.head()

## Extract day of week from visit_date

In [ ]:
train['weekday'] = train.visit_date.apply(lambda t: t.dayofweek)

In [ ]:
train.head()

In [ ]:
test['weekday'] = test.visit_date.apply(lambda t: t.dayofweek)

In [ ]:
test.head()

## Train / validation split and column dropping

In [ ]:
validation = train[train.week_count > 60]

In [ ]:
train_val = train[train.week_count <= 60]

In [ ]:
from sklearn.utils import shuffle
train =  shuffle(train, random_state=21)


X_train, X_valid = train_test_split(train, test_size=0.05, random_state=42)


In [ ]:
print (X_train.shape, X_valid.shape, test.shape)

In [ ]:
col_to_drop = ['air_store_id', 'visit_date', 'air_genre_name', 'air_area_name', 'calendar_date']

In [ ]:
test_id = test.pop('id')
test.drop('visitors', axis=1, inplace=True)
test.drop(col_to_drop, axis=1, inplace=True)

In [ ]:
train_labels = train.pop('visitors')
train.drop(col_to_drop, axis=1, inplace=True)

In [ ]:
y_train = X_train.pop('visitors')
X_train.drop(col_to_drop, axis=1, inplace=True)

In [ ]:
y_valid = X_valid.pop('visitors')
X_valid.drop(col_to_drop, axis=1, inplace=True)

In [ ]:
model = LGBMRegressor()



In [ ]:
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor()



In [ ]:
#model.fit(X_train, y_train)

val_pred = model.predict(X_valid)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
print ('Validation RMSLE: ', mean_squared_error(np.log1p(y_valid),np.log1p(val_pred))**0.5)

## Parameter tuning

In [ ]:
neighbors = list(xrange(1,30))

train_results = []
test_results = []
for n in neighbors:
    model = KNeighborsRegressor(n_neighbors=n)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_valid)
    
    test_results.append(mean_squared_error(np.log1p(y_valid),np.log1p(y_pred))**0.5)

In [ ]:
from matplotlib.legend_handler import HandlerLine2D


line2, = plt.plot(neighbors, test_results, 'r', label="Validation RMSE")

plt.legend(handler_map={line2: HandlerLine2D(numpoints=1)})

plt.ylabel('RMSEL')
plt.xlabel('n_neighbors')
plt.show()

In [ ]:
model = KNeighborsRegressor(n_neighbors=8)

In [ ]:
model.fit(X_train, y_train)

val_pred = model.predict(X_valid)

In [ ]:
print ('Validation RMSLE: ', mean_squared_error(np.log1p(y_valid),np.log1p(val_pred))**0.5)

In [ ]:


model.fit(train, train_labels)

train_pred = model.predict(train)

print ('Train RMSLE: ', mean_squared_error(np.log1p(train_labels), np.log1p(train_pred))**0.5)

test_pred = model.predict(test)

In [ ]:
test_submit = pd.DataFrame({'id':test_id, 'visitors': test_pred})
print test_submit.shape
test_submit.to_csv('merge.csv', index=False)
test_submit.head(10)

In [ ]:
test_submit.visitors.describe()